# (MBTI 별 Data 모으기: 생략) MBTI file 불러오기

In [1]:
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from collections import Counter

In [2]:
mbti=pd.read_csv('.\data\\training\\mbti.csv')

# Tokenize and Stemming


In [3]:
X_data = mbti['posts'].ravel()
y_data = mbti['type'].ravel()

print("normalize 시작")
normalized_text = []
for sentence in X_data: #for sentence in X_data.ravel()
    clean_sentence = re.sub('[^A-Za-z\s]+', '', sentence.lower())
    normalized_text.append(clean_sentence)
X_data = normalized_text
print("normalize 완료")

normalize 시작
normalize 완료


In [4]:
import nltk
# nltk.download('punkt')
# nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
ps=PorterStemmer()

# print("stopwords 제거 시작")
stopwords = ['http*']
#print("stopwords 제거 완료")

print("stemming 시작")
stemData=[]
for sentence in X_data:
    tokenData = nltk.word_tokenize(sentence)
    tempData = []
    for word in tokenData:
        tempData.append(ps.stem(word))
        #tempData = [word for word in tempData if not word in stopwords] # 불용어 제거
    stemData.append(tempData)
print("stemming 완료")
#print(stemData[0])

#'list' object has no attribute 'lower' 문제 발생
#2차원 리스트인 stemData를 1차원 리스트로
#flat_stem = [item for sublist in stemData for item in sublist]

stemming 시작
stemming 완료


# Garbage Collection 1

In [6]:
import gc

'''del [[mbti]]

del sentence

del [[dataCombine]]

del [condition]

for ownsentence in allData:
    del ownsentence
del allData

for tokenData in tokenizedData:   # delete 'stemData'
    for data in tokenData:
        del data
    del tokenData
del tokenizedData'''

for clean_sentence in normalized_text:
    del clean_sentence
del normalized_text

del X_data

del tokenData
del tempData

del sentence

gc.collect()

139

# Vectorization (tfidf)

In [7]:
def dummy_fun(doc):
    return doc

tfidf=TfidfVectorizer(
    analyzer='word',
    tokenizer=dummy_fun,
    preprocessor=dummy_fun,
    token_pattern=None
)

tfidf.fit(stemData)
X = tfidf.transform(stemData)
print(X)

print(y_data)
y = np.array(y_data)
print(y)

  (0, 294178)	0.018313994351578357
  (0, 293671)	0.05667299274712749
  (0, 292648)	0.14736037481992406
  (0, 291475)	0.012844119309787095
  (0, 291298)	0.015917293926600354
  (0, 289803)	0.06245131892979903
  (0, 288699)	0.018554642544113317
  (0, 287300)	0.08606730960753338
  (0, 286757)	0.017381440260737353
  (0, 285917)	0.028056984359135558
  (0, 285593)	0.1016963400988987
  (0, 285472)	0.01968822480951418
  (0, 284394)	0.011231264358716912
  (0, 283395)	0.010640763011372371
  (0, 282902)	0.024391435899897337
  (0, 282448)	0.03860584354908418
  (0, 281916)	0.022327030698545473
  (0, 281840)	0.009444409717358306
  (0, 281370)	0.03597244101296453
  (0, 281297)	0.02120252125544941
  (0, 280685)	0.011532506841579923
  (0, 280567)	0.05811753463491229
  (0, 280476)	0.04925191972988302
  (0, 279481)	0.010691768530648934
  (0, 278952)	0.03210151554628096
  :	:
  (8674, 15476)	0.01889596232388363
  (8674, 14739)	0.0730334088398339
  (8674, 13090)	0.009365984799265245
  (8674, 11910)	0.022782

In [8]:
'''print(tfidf_matrix.shape)

max_words = max(len(l) for l in X)   # post의 최대 길이(단어 수)
print(max_words)
'''print('post의 평균 길이 :{}'.format(sum(map(len, stemData))/len(stemData)))
print('post의 전체 길이 :{}'.format(sum(map(len, stemData))))'''
post_num = len(stemData)    #post의 갯수
print(post_num)'''

SyntaxError: invalid syntax (<ipython-input-8-3c22e5f7321d>, line 5)

# Garbage Collection

In [9]:
import psutil
# import os
# import gc

In [10]:
for data in stemData:   # delete 'stemData'
    for component in data:
        del component
    del data
del stemData

gc.collect()

614

In [11]:
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1234)

model = Sequential()
model.add(Embedding(max_words, post_num))
model.add(LSTM(128))
model.add(Dense(16, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split = 0.2)

ValueError: Please provide as model inputs either a single array or a list of arrays. You passed: inputs=  (0, 293671)	0.02955070789316349
  (0, 292648)	0.09796763951958413
  (0, 292429)	0.039119493552237324
  (0, 292224)	0.012193632747822246
  (0, 291475)	0.01607338378879723
  (0, 291298)	0.01991921501121431
  (0, 291185)	0.03318935944756016
  (0, 291156)	0.03679040708606734
  (0, 289363)	0.033682024142516705
  (0, 288699)	0.011609822498621039
  (0, 288529)	0.024932895368988028
  (0, 287950)	0.009637792632344915
  (0, 287427)	0.00753071391516984
  (0, 285593)	0.07520192861562311
  (0, 284394)	0.014055025347945704
  (0, 283536)	0.01221605924451546
  (0, 283395)	0.013316060335651062
  (0, 283052)	0.009222820738824754
  (0, 282903)	0.024077541609769052
  (0, 282902)	0.02289294234133414
  (0, 282725)	0.05385316289988784
  (0, 282720)	0.025848888602759232
  (0, 282448)	0.0724681690766283
  (0, 281840)	0.02954730067208133
  (0, 281370)	0.007502770155945566
  :	:
  (6071, 13090)	0.0152229120

In [15]:
print(X_train.toarray()[0])

[0.25869917 0.         0.         ... 0.         0.         0.        ]


In [14]:
X_train.shape

(6072, 295647)

In [ ]:
model.save('.\\models\\LSTM_undersample.h5')

In [ ]:
model.summary()

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['acc'])
plt.plot(epochs, history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
epochs = range(1, len(history.history['acc']) + 1)
plt.plot(epochs, history.history['loss'])
plt.plot(epochs, history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
loss_and_metrics = model.evaluate(X_test, y_test, batch_size=32)